In [4]:
import psycopg2
import pandas as pd
from psycopg2 import extras

In [9]:
# Database connection details 
db_name = "postgres"
db_user = "postgres"
db_password = "Harsh@1234" 
db_host = "127.0.0.1"  # Assuming PostgreSQL is running on the docker machine
# Table name
table_name = "sales_data1"

In [10]:
# Define error handling function for file loading
def handle_file_loading_error(filename):
    try:
        data = pd.read_csv(filename,delimiter=',')
        return data
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found. Please check the file path.")
    except pd.errors.ParserError as e:
        print(f"Error parsing CSV file '{filename}': {e}")
    return None  # Return None to indicate an error occurred

# Load the CSV data with error handling
data = handle_file_loading_error("sales_data_cleaned.csv")
print(data.head())  # Display the first few rows

   TransactionID  ProductID  Quantity  SalePrice  PurchasePrice
0              1       1124         5      13.38           9.91
1              2       1146        14      26.88          25.60
2              3       1128         8      15.89          13.24
3              4       1101         8       2.90           2.42
4              5       1112         2       7.23           4.82


In [11]:
#dataset meta-information
nomenclature=pd.DataFrame(data.info()).filter(like='Non-Null Count').iloc[:, :-2]
print(nomenclature)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2566 entries, 0 to 2565
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TransactionID  2566 non-null   int64  
 1   ProductID      2566 non-null   int64  
 2   Quantity       2566 non-null   int64  
 3   SalePrice      2566 non-null   float64
 4   PurchasePrice  2566 non-null   float64
dtypes: float64(2), int64(3)
memory usage: 100.4 KB
Empty DataFrame
Columns: []
Index: []


In [12]:


# Assuming the following variables are defined:
# db_user, db_password, db_host, db_name, table_name, page_size

def transfer_sales_data(csv_file):
    """
    Transfers data from a CSV file to the sales database table.

    Args:
        csv_file (str): Path to the CSV file containing sales data.
    """
    connection = psycopg2.connect(user=db_user, password=db_password, host=db_host, port=5432, dbname=db_name)
    cursor = connection.cursor()

    try:
        # Create table (if it doesn't exist)
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            TransactionID BIGINT PRIMARY KEY,  /* Assuming TransactionID is the primary key */
            ProductID INT,
            Quantity INT,
            SalePrice DECIMAL(10, 2),  /* Assuming 2 decimal places for SalePrice */
            PurchasePrice DECIMAL(10, 2)   /* Assuming 2 decimal places for PurchasePrice */
        );
        """
        cursor.execute(create_table_query)
        connection.commit()
        print("Table created successfully.")

    except (Exception, psycopg2.Error) as error:
        print("Error creating table:", error)
        connection.rollback()  # Rollback on error

    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv(csv_file)

    try:
        # Create a list of tuples from the dataframe values
        tuples = [tuple(x) for x in df.to_numpy()]
        # Comma-separated dataframe columns
        cols = ','.join(list(df.columns))
        # SQL query to execute
        query = f"INSERT INTO {table_name}({cols}) VALUES(%s,%s,%s,%s,%s)"
        extras.execute_batch(cursor, query, tuples, page_size=1000)
        connection.commit()
        print("execute_batch() done")

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        connection.rollback()
        cursor.close()
        return 1

    cursor.close()

# Example usage of the function
try:
    transfer_sales_data("sales_data_cleaned.csv")
except Exception as e:
    print(f"An error occurred: {e}")


Table created successfully.
execute_batch() done
